# Merging Data Frames

The Merging stage is an operation at the data frame level (not a cell operation). 

The merging process combines **TWO** data frames, if and only if, they each have a column that is _technically_ the same (both match). Unmatched values will not be part of the output. If you have messy data, you need to clean at least those **key** columns for the match to work.

Let's see some data. First, a previously  formatted data from this webpage:

In [ ]:
%%html
<iframe width="700" height="300" src="https://en.wikipedia.org/wiki/List_of_freedom_indices" allowfullscreen></iframe>


Let's use the pickle version:

In [ ]:
import pandas as pd
import os


allFree=pd.read_pickle(os.path.join("DataFiles","allFree.pkl") )  

Now, let's bring this other one:

In [ ]:
%%html

<iframe width="700" height="300" src="https://www.cia.gov/the-world-factbook/field/military-expenditures/country-comparison" allowfullscreen></iframe>


In [ ]:
linkCIA="https://www.cia.gov/the-world-factbook/field/military-expenditures/country-comparison"
mil=pd.read_html(linkCIA,flavor='bs4')
len(mil)

In [ ]:
# the only one
mil[0]

Let's check format:

In [ ]:
mil[0].info()

Let's keep the columns we need from the data frame:

In [ ]:
mil[0]=mil[0].iloc[:,[1,2]]
mil[0].head()

Let's create a new data frame, while renaming the second column with a simpler name:

In [ ]:
mili=mil[0].rename(columns={"% of GDP": "mili_pctGDP"})
mili.head()

## Checking amount of matches

Let's just do the merge and check how many rows resulted. The common column we need for the matching is _Country_ in both data frames.


In [ ]:
allFree.shape , mili.shape,allFree.merge(mili).shape

The **mili** data frame has 166 rows, so we will not be able to get 197 rows, the amount of rows in _allFree_. However, we just matched 159. 

In this situation, let's see if we can recover more matches.

In [ ]:
# I will try to recover some of these
InMiliNotInFree=list(set(mili.Country)-set(allFree.Country))
InMiliNotInFree

In [ ]:
# I hope this countries can match
InFreeUnmatched=list(set(allFree.Country)-set(mili.Country))
InFreeUnmatched

## Fyzzy matching

Let's try to match strings that are NOT equally written. You need to previously install:
* thefuzz (use _pip install thefuzz_)
* python-Levenshtein  (use _pip python-Levenshtein_)

In [ ]:
from thefuzz import process

[(country, process.extractOne(country, InFreeUnmatched)) for country in sorted(InMiliNotInFree)]

Great!...we found good matches. Let's build a dictionary:

In [ ]:
changesMili={country: process.extractOne(country, InFreeUnmatched)[0] for country in sorted(InMiliNotInFree)}
changesMili

In [ ]:
mili.Country.replace(changesMili,inplace=True)

Let's recheck the amount of matches:

In [ ]:
allFree.shape , mili.shape,allFree.merge(mili).shape

Let me create the new data frame, being a little over explicit:

In [ ]:
# in case you have different names in the matching columns:
freemili=allFree.merge(mili,left_on='Country', right_on='Country')
freemili

We can save this for R and Python :

In [ ]:
# for Python

import os
freemili.to_pickle(os.path.join("DataFiles","mergedData.pkl"))

In [ ]:
# for R

from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(freemili,file=os.path.join('DataFiles','mergedData.RDS'))